# JSON 데이터와 데이터프레임


**[목차]**
- JSON to DataFrame
- DataFrame to JSON

## 1. JSON to DataFrame
`Pandas 라이브러리`를 사용하면 JSON 파일을 데이터프레임(DataFrame)으로 변환할 수 있음 </br>
변환된 데이터프레임을 이용하면 다양한 데이터 분석 및 시각화 라이브러리가 이용 가능하여 효과적인 데이터 분석 가능

In [ ]:
import json
import pandas as pd

### 1.1 json.load로 불러오기

In [ ]:
# target.json 파일을 읽어 Python 객체로 변환
with open('data/target.json') as json_file :
    load_data = json.load(json_file)

In [ ]:
# load한 json 데이터 확인
print(load_data)

In [ ]:
print(type(load_data))

In [ ]:
# load한 json 데이터 dumps로 확인
print(json.dumps(load_data, indent=4))

In [ ]:
load_df1 = pd.DataFrame(load_data)
load_df1

### 1.2 pd.read_json 으로 불러오기

In [ ]:
load_df2 = pd.read_json('data/target.json')
load_df2

현재 load_df2는 정상적으로 분석하기가 어렵기 때문에 `json_normalize`라는 메소드를 이용

`json_normalize`를 사용하면 JSON 객체 또는 JSON으로 구성된 리스트의 중첩된 구조를 평면화(flat)하여 pandas DataFrame으로 변환

In [ ]:
pd.json_normalize(load_df2["employee"])

### 1.3 불러온 데이터프레임으로 데이터 분석하기
employee에서 **skill의 빈도수를 도출**하는 간단한 데이터 분석을 진행

In [ ]:
df = pd.DataFrame(load_data["employee"])
df

In [ ]:
df = pd.json_normalize(load_df2["employee"])
df

하지만 skills 컬럼의 데이터는 리스트 형태로 저장되어 있어서 개별적인 분석이 어려움

In [ ]:
df['skills']

Pandas의 `explode 메소드`를 사용하면 리스트로 저장된 데이터를 각 기술을 별도의 행으로 분리할 수 있음

In [ ]:
# explode 확인을 위한 샘플 데이터 생성  
sample_df = pd.DataFrame({  
    'A': [1, 2, 3],  
    'B': [['X', 'Y'], ['X', 'Z'], ['W']],  
    'C': [['Apple', 'Banana'], ['Grape'], ['Cherry', 'Date']]  
})  

print("Original DataFrame:")  
print(sample_df)  

In [ ]:
# 'B' 컬럼을 기준으로 explode 실행  
df_exploded_B = sample_df.explode('B')  

print("\nDataFrame after exploding 'B' column:")  
print(df_exploded_B)  

In [ ]:
# 'C' 컬럼을 기준으로 explode 실행  
df_exploded_C = sample_df.explode('C')  

print("\nDataFrame after exploding 'C' column:")  
print(df_exploded_C)  

`explode 메소드`를 사용하여 skills 컬럼을 재구성

In [ ]:
df_skills = df['skills'].explode()
df_skills

In [ ]:
# 각 기술의 빈도수 계산
skills_frequency_df = df_skills.value_counts().reset_index()
skills_frequency_df

In [ ]:
# 열 이름을 명확하게 변경
skills_frequency_df.columns = ['Skill', 'Frequency']  
skills_frequency_df

추출한 데이터프레임을 이용하여 데이터를 시각화하는 과정 진행

In [ ]:
import matplotlib.pyplot as plt

# 데이터프레임으로부터 막대 그래프를 생성
plt.figure(figsize=(10, 6))

plt.bar(skills_frequency_df['Skill'], 
        skills_frequency_df['Frequency'], 
        color='skyblue'
        )

# 제목과 라벨을 추가
plt.title('Frequency of Skills')
plt.xlabel('Skill')
plt.ylabel('Frequency')

# x축에 있는 기술 이름을 회전시켜 가독성 상승
plt.xticks(rotation=45)

# 그래프를 표시
# 회전된 x축 라벨을 위해 레이아웃을 조정
plt.tight_layout()  
plt.show()


## 2. DataFrame to JSON
Pandas 라이브러리에서 지원하는 `to_json 함수`를 사용하면 분석한 데이터프레임을 JSON으로 저장할 수 있음 </br>
단, Series 자료형은 JSON으로 저장할 수 없음

### 2.1 새로운 데이터 추가하기

In [ ]:
new_employee = [{
        "id" : 108, 
        "name" : "King", 
        "department" : "Developer", 
        "skills" : ["leadership", "communication"]
    },
    {
        "id" : 133, 
        "name" : "KIM", 
        "department" : "Marketer", 
        "skills" : ["teamwork", "decisiveness"]
    }]


new_df = pd.DataFrame(new_employee)
new_df

In [ ]:
df

In [ ]:
df_concat = pd.concat([df, new_df], ignore_index=True)
df_concat

In [ ]:
# to_json 함수를 이용해서 저장
df_concat.to_json("employee_data(records).json", orient='records')

In [ ]:
# employee_data.json 파일을 읽어서 확인하기
with open('employee_data(records).json') as employee_data :
    load_data = json.load(employee_data)

print(json.dumps(load_data, indent=4))

저장하고자 하는 형식에 따라 `to_json`의 orient 옵션을 변경함으로써 사용자가 원하는 대로 json 데이터를 저장할 수 있음


| orient 옵션 | 설명 |
| ----- | ----- | 
|split |데이터를 객체로 분리하여 저장합니다. 예를 들어, 컬럼 이름, 데이터, 인덱스가 별도의 키로 저장됩니다 |
|records | 각 행을 별도의 JSON 객체로 변환합니다. 컬럼 이름을 키로 하고 각 행의 값들을 값으로 사용합니다 |
|index | 인덱스를 키로 사용하고 각 행의 데이터를 값으로 사용합니다. |
|columns | 컬럼을 키로 사용하고 각 컬럼의 데이터를 저장합니다. orient 옵션을 지정하지 않으면 columns가 기본(default)으로 설정됩니다 |
|values | 데이터를 순수한 2D 배열로 변환합니다. |
|table | 데이터를 추가 메타데이터와 함께 JSON 테이블 스키마 형식으로 변환합니다. |

In [ ]:
df_concat.to_json("employee_data(split).json", orient='split')

with open('employee_data(split).json') as employee_data :
    load_data = json.load(employee_data)

print(json.dumps(load_data, indent=4))

In [ ]:
df_concat.to_json("employee_data(index).json", orient='index')

with open('employee_data(index).json') as employee_data :
    load_data = json.load(employee_data)

print(json.dumps(load_data, indent=4))

In [ ]:
df_concat.to_json("employee_data(columns).json", orient='columns')

with open('employee_data(columns).json') as employee_data :
    load_data = json.load(employee_data)

print(json.dumps(load_data, indent=4))

In [ ]:
df_concat.to_json("employee_data(values).json", orient='values')

with open('employee_data(values).json') as employee_data :
    load_data = json.load(employee_data)

print(json.dumps(load_data, indent=4))

In [ ]:
df_concat.to_json("employee_data(table).json", orient='table')

with open('employee_data(table).json') as employee_data :
    load_data = json.load(employee_data)

print(json.dumps(load_data, indent=4))

### [TODO] JSON 파일을 데이터프레임으로 기반으로 파싱하기
* data 폴더에 있는 `parsing_target.json` 은 자동차 주행 중에 센서로 습득한 데이터의 일부분입니다.
* 해당 데이터를 읽고 points 데이터를 추출하여 x, y 로 구분하는 코드를 작성하세요.
* 구분된 데이터는 `result.csv`로 저장하는 코드를 작성하세요.

먼저 json 데이터를 확인하겠습니다. 

In [ ]:
with open('data/parsing_target.json') as json_file :
    load_data = json.load(json_file)

print(json.dumps(load_data, indent=4))

Pandas의 `read_json` 을 이용해서 JSON 파일을 데이터프레임으로 변환해 보세요.

In [ ]:
target_df = None
target_df

현재의 데이터프레임으로 데이터 분석을 하기에는 무리가 있어 보입니다. 

마찬가지로 Pandas의 `json_normalize` 를 이용하여 현재의 데이터프레임을 분석하기 좋도록 변환해 보겠습니다.

In [ ]:
annotations = None
annotations.head()

확인해본 결과, points 데이터는 리스트 안에 리스트가 있는 **2중 리스트** 형태로 저장되어 있습니다. 

앞에서 배운 `explode` 메소드를 이용해서 데이터를 분할하고 그 결과를 확인해 보겠습니다.

In [ ]:
points = None
points

`explode` 메소드를 통해 분할한 결과, 원하는 결과로 분할이 되었습니다.

이제 이 데이터를 x, y 컬럼을 가진 데이터프레임으로 저장하겠습니다. 이를 위해 분할된 데이터를 리스트 형태로 변경하는 코드를 작성해 주세요.

In [ ]:
points_list = None
points_list

이제 이 데이터를 "x" 와 "y" 컬럼을 가진 데이터프레임으로 만드는 코드를 작성하세요.

In [ ]:
result_df = None
result_df

마지막으로 지금까지 다룬 데이터프레임을 `result.csv` 파일로 저장하는 코드를 작성해 주세요.

csv 파일로 저장하는 함수에 `index=False` 인자를 추가하면 불필요한 인덱스를 제거할 수 있습니다. 

이를 참고하여 csv 파일을 생성하세요. 파일 경로가 달라지면 채점이 정상적으로 동작하지 않을 수 있습니다.

In [ ]:
None